In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/diabetes-prediction-dataset/diabetes_prediction_dataset.csv


In [2]:
sugar = pd.read_csv("/kaggle/input/diabetes-prediction-dataset/diabetes_prediction_dataset.csv")
sugar.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [3]:
sugar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   gender               100000 non-null  object 
 1   age                  100000 non-null  float64
 2   hypertension         100000 non-null  int64  
 3   heart_disease        100000 non-null  int64  
 4   smoking_history      100000 non-null  object 
 5   bmi                  100000 non-null  float64
 6   HbA1c_level          100000 non-null  float64
 7   blood_glucose_level  100000 non-null  int64  
 8   diabetes             100000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 6.9+ MB


In [4]:
sugar.describe()

,age,hypertension,heart_disease,bmi,HbA1c_level,blood_glucose_level,diabetes
count,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000
mean,41.885856,0.07485,0.039420,27.320767,5.527507,138.058060,0.085000
std,22.516840,0.26315,0.194593,6.636783,1.070672,40.708136,0.278883
min,0.080000,0.00000,0.000000,10.010000,3.500000,80.000000,0.000000
25%,24.000000,0.00000,0.000000,23.630000,4.800000,100.000000,0.000000
50%,43.000000,0.00000,0.000000,27.320000,5.800000,140.000000,0.000000
75%,60.000000,0.00000,0.000000,29.580000,6.200000,159.000000,0.000000
max,80.000000,1.00000,1.000000,95.690000,9.000000,300.000000,1.000000


In [5]:
sugar.describe(include="O")

,gender,smoking_history
count,100000,100000
unique,3,6
top,Female,No Info
freq,58552,35816


# Data Preprocessing

## Null Value Analysis

In [6]:
sugar.isna().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

Since no Null Values are Present, we'll skip this step

## Correcting DataType

In [7]:
sugar.age = sugar.age.astype(int)

## Label Encoding

In [8]:
from sklearn.preprocessing import LabelEncoder

sugar[["gender", "smoking_history"]] = sugar[["gender", "smoking_history"]].apply(LabelEncoder().fit_transform)

In [9]:
sugar.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,0,80,0,1,4,25.19,6.6,140,0
1,0,54,0,0,0,27.32,6.6,80,0
2,1,28,0,0,4,27.32,5.7,158,0
3,0,36,0,0,1,23.45,5.0,155,0
4,1,76,1,1,1,20.14,4.8,155,0


# Model Training

In [10]:
X = sugar.drop("diabetes", axis = 1)
y = sugar.diabetes

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [12]:
from sklearn.metrics import f1_score, accuracy_score, mean_squared_error, classification_report, confusion_matrix

## Decision Tree Classifier

In [13]:
from sklearn.tree import DecisionTreeClassifier

dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train, y_train)
y_pred_dt = dt_classifier.predict(X_test)

In [14]:
print("Accuracy Score: ", accuracy_score(y_test, y_pred_dt) * 100)
print("\n\nMean Squared Error: ", mean_squared_error(y_test, y_pred_dt))
print("\n\nF1 Score: ", f1_score(y_test, y_pred_dt))
print("\n\nClassification Report\n", classification_report(y_test, y_pred_dt))
print("\n\nConfusion Matris\n", confusion_matrix(y_test, y_pred_dt))

Accuracy Score:  95.075


Mean Squared Error:  0.04925


F1 Score:  0.719293245939014


Classification Report
               precision    recall  f1-score   support

           0       0.98      0.97      0.97     18297
           1       0.70      0.74      0.72      1703

    accuracy                           0.95     20000
   macro avg       0.84      0.86      0.85     20000
weighted avg       0.95      0.95      0.95     20000



Confusion Matris
 [[17753   544]
 [  441  1262]]


## RandomForest Classifier

In [15]:
from sklearn.ensemble import RandomForestClassifier

rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(X_train, y_train)
y_pred_rf = rf_classifier.predict(X_test)

In [16]:
print("Accuracy Score: ", accuracy_score(y_test, y_pred_rf) * 100)
print("\n\nMean Squared Error: ", mean_squared_error(y_test, y_pred_rf))
print("\n\nF1 Score: ", f1_score(y_test, y_pred_rf))
print("\n\nClassification Report\n", classification_report(y_test, y_pred_rf))
print("\n\nConfusion Matris\n", confusion_matrix(y_test, y_pred_rf))

Accuracy Score:  97.085


Mean Squared Error:  0.02915


F1 Score:  0.801498127340824


Classification Report
               precision    recall  f1-score   support

           0       0.97      1.00      0.98     18297
           1       0.95      0.69      0.80      1703

    accuracy                           0.97     20000
   macro avg       0.96      0.84      0.89     20000
weighted avg       0.97      0.97      0.97     20000



Confusion Matris
 [[18240    57]
 [  526  1177]]


## XG Bosst

In [17]:
from xgboost import XGBClassifier

xg_classifier = XGBClassifier(learning_rate = 0.5, n_estimators = 100)
xg_classifier.fit(X_train, y_train)
y_pred_xg = xg_classifier.predict(X_test)

In [18]:
print("Accuracy Score: ", accuracy_score(y_test, y_pred_xg) * 100)
print("\n\nMean Squared Error: ", mean_squared_error(y_test, y_pred_xg))
print("\n\nF1 Score: ", f1_score(y_test, y_pred_xg))
print("\n\nClassification Report\n", classification_report(y_test, y_pred_xg))
print("\n\nConfusion Matris\n", confusion_matrix(y_test, y_pred_xg))

Accuracy Score:  97.03500000000001


Mean Squared Error:  0.02965


F1 Score:  0.8017385489802742


Classification Report
               precision    recall  f1-score   support

           0       0.97      1.00      0.98     18297
           1       0.93      0.70      0.80      1703

    accuracy                           0.97     20000
   macro avg       0.95      0.85      0.89     20000
weighted avg       0.97      0.97      0.97     20000



Confusion Matris
 [[18208    89]
 [  504  1199]]


## Cat Boost

In [19]:
from catboost import CatBoostClassifier

cb_classifier = CatBoostClassifier(learning_rate=0.5, n_estimators=100)
cb_classifier.fit(X_train, y_train)

0:	learn: 0.3547696	total: 70.4ms	remaining: 6.96s
1:	learn: 0.2304813	total: 78.3ms	remaining: 3.84s
2:	learn: 0.1706219	total: 87ms	remaining: 2.81s
3:	learn: 0.1342434	total: 94.9ms	remaining: 2.28s
4:	learn: 0.1140619	total: 103ms	remaining: 1.95s
5:	learn: 0.1015065	total: 111ms	remaining: 1.73s
6:	learn: 0.0958088	total: 119ms	remaining: 1.58s
7:	learn: 0.0906094	total: 127ms	remaining: 1.47s
8:	learn: 0.0870270	total: 135ms	remaining: 1.37s
9:	learn: 0.0857500	total: 143ms	remaining: 1.28s
10:	learn: 0.0839062	total: 151ms	remaining: 1.22s
11:	learn: 0.0823687	total: 159ms	remaining: 1.16s
12:	learn: 0.0819095	total: 166ms	remaining: 1.11s
13:	learn: 0.0812076	total: 174ms	remaining: 1.07s
14:	learn: 0.0805610	total: 181ms	remaining: 1.02s
15:	learn: 0.0802224	total: 188ms	remaining: 989ms
16:	learn: 0.0798561	total: 196ms	remaining: 957ms
17:	learn: 0.0795753	total: 204ms	remaining: 927ms
18:	learn: 0.0791492	total: 211ms	remaining: 898ms
19:	learn: 0.0789402	total: 218ms	remai

In [20]:
y_pred_cb = cb_classifier.predict(X_test)

In [21]:
print("Accuracy Score: ", accuracy_score(y_test, y_pred_cb) * 100)
print("\n\nMean Squared Error: ", mean_squared_error(y_test, y_pred_cb))
print("\n\nF1 Score: ", f1_score(y_test, y_pred_cb))
print("\n\nClassification Report\n", classification_report(y_test, y_pred_cb))
print("\n\nConfusion Matris\n", confusion_matrix(y_test, y_pred_cb))

Accuracy Score:  97.16


Mean Squared Error:  0.0284


F1 Score:  0.8068027210884353


Classification Report
               precision    recall  f1-score   support

           0       0.97      1.00      0.98     18297
           1       0.96      0.70      0.81      1703

    accuracy                           0.97     20000
   macro avg       0.97      0.85      0.90     20000
weighted avg       0.97      0.97      0.97     20000



Confusion Matris
 [[18246    51]
 [  517  1186]]
